# Taylor expansions

__Zoïs Moitier__

_Karlsruhe Institute of Technology, Germany_

---

## Internals

In [1]:
import sympy as sy
from IPython.display import display

import src

In [2]:
ζ, η, X = sy.symbols("zeta eta X")
order = 12

In [3]:
def taylor_w(order):
    if order <= 2:
        return 1 - 2 ** sy.Rational(-1, 3) * ζ + sy.O(ζ**order)

    w = 1 - 2 ** sy.Rational(-1, 3) * ζ
    α = sy.symbols("alpha")
    for k in range(2, order):
        w += α * ζ**k
        expr_l = (ζ * w**2 + sy.O(ζ ** (k + 1))).expand()
        expr_r = ((1 - w**2) * w.diff(ζ) ** 2 + sy.O(ζ ** (k + 1))).expand()
        eq = sy.Eq(expr_l.coeff(ζ, k), expr_r.coeff(ζ, k))
        w = w.subs(α, sy.solveset(eq, α).args[0])

    return w + sy.O(ζ**order)

In [4]:
def _calc_B(ψ, A, k):
    expr_r = (ψ * A[k] - A[k].diff(ζ, 2)).expand()

    bn = sy.symbols(f"b0:{expr_r.getn()}")
    Bk = sum([s * ζ**n for n, s in enumerate(bn)], expr_r.getO())
    expr_l = (2 * ζ * Bk.diff(ζ) + Bk).expand()

    return sum(
        [
            (expr_r.coeff(ζ, n) / expr_l.coeff(ζ, n).coeff(bn[n])) * ζ**n
            for n in range(expr_r.getn())
        ],
        expr_r.getO(),
    )


def _calc_A(A, B, k):
    Ak = 0
    for n in range(1, k):
        Ak -= (A[n] * A[k - n]).expand()
    for n in range(k):
        Ak -= (A[n] * B[k - 1 - n].diff(ζ)).expand()
        Ak += (A[n].diff(ζ) * B[k - 1 - n]).expand()
        Ak += (ζ * B[n] * B[k - 1 - n]).expand()
    return (Ak / 2).expand()


def taylor_AB(ψ, K):
    A = [1 + 0 * ζ]
    B = [_calc_B(ψ, A, 0)]

    for k in range(1, K + 1):
        A.append(_calc_A(A, B, k))
        B.append(_calc_B(ψ, A, k))

    return (A, B)

In [5]:
def taylor_CD(χ, A, B, K):
    C = [(χ * A[k] + A[k].diff(ζ) + ζ * B[k]).expand() for k in range(K + 1)]
    D = [1 + 0 * ζ]
    D.extend(
        [(A[k] + χ * B[k - 1] + B[k - 1].diff(ζ)).expand() for k in range(1, K + 1)]
    )

    return (C, D)

---

## Taylor expansion of $w(\zeta)$

In [6]:
w = taylor_w(order)
# display(w.subs(ζ, 2 ** sy.Rational(1, 3) * η))
display(w)

1 - 2**(2/3)*zeta/2 + 3*2**(1/3)*zeta**2/20 + zeta**3/700 - 479*2**(2/3)*zeta**4/252000 - 20231*2**(1/3)*zeta**5/32340000 - 171389*zeta**6/4204200000 + 203448767*2**(2/3)*zeta**7/7945938000000 + 4236299281*2**(1/3)*zeta**8/360215856000000 + 750913741013*zeta**9/627375949200000000 - 11800431220250863*2**(2/3)*zeta**10/21343329791784000000000 - 632807379979991399*2**(1/3)*zeta**11/2127218535914472000000000 + O(zeta**12)

---

## Taylor expansion $\phi_0(\zeta)$

In [7]:
ϕ0 = ((((1 - w**2) / (4 * ζ)).expand()) ** sy.Rational(-1, 4)).series(
    ζ, x0=0, n=order - 1
)
# display((ϕ0.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
display(ϕ0)
# print([src.num2str(ϕ0.coeff(ζ, k)) for k in range(order - 1)])

2**(1/3) + zeta/5 + 9*2**(2/3)*zeta**2/700 - 89*2**(1/3)*zeta**3/31500 - 4547*zeta**4/2310000 - 18251*2**(2/3)*zeta**5/140140000 + 52621141*2**(1/3)*zeta**6/567567000000 + 181116587*zeta**7/3411135000000 + 240938108053*2**(2/3)*zeta**8/89625135600000000 - 17587597826723657*2**(1/3)*zeta**9/5335832447946000000000 - 69206978393163893*zeta**10/39392935850268000000000 + O(zeta**11)

---

## Taylor expansion $\phi_1(\zeta)$

In [8]:
ϕ1 = (
    (2 / w).series(ζ, n=order)
    * ((((1 - w**2) / (4 * ζ)).expand()) ** sy.Rational(1, 4)).series(ζ, n=order - 1)
).expand()
# display((ϕ1.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(2, 3)).expand())
display(ϕ1)
# print([src.num2str(ϕ1.coeff(ζ, k)) for k in range(order - 1)])

2**(2/3) + 4*2**(1/3)*zeta/5 + 18*zeta**2/35 + 44*2**(2/3)*zeta**3/315 + 39793*2**(1/3)*zeta**4/606375 + 591156*zeta**5/21896875 + 304382131*2**(2/3)*zeta**6/62077640625 + 21766532*2**(1/3)*zeta**7/13705453125 + 1370455137*zeta**8/2932446343750 + 105807812767*2**(2/3)*zeta**9/1636035753515625 + 1917245573688890701*2**(1/3)*zeta**10/109061497202646269531250 + O(zeta**11)

---

## Taylor expansion $\psi(\zeta)$

In [9]:
ψ = (
    5 / (16 * ζ**2)
    + (
        w**2
        * (w**2 + 4)
        * (1 / (4 * (w**2 - 1) ** 3 / ζ**3).expand()).series(ζ, n=order - 1)
        / ζ**2
    ).expand()
)
# display((ψ.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
display(ψ)
# print([src.num2str(ψ.coeff(ζ, k)) for k in range(order - 1)])

2**(1/3)/70 + 2*zeta/75 + 69*2**(2/3)*zeta**2/13475 - 148*2**(1/3)*zeta**3/73125 - 19232*zeta**4/7074375 - 29368*2**(2/3)*zeta**5/72515625 + 1208831912*2**(1/3)*zeta**6/6986122171875 + 49058816*zeta**7/251266640625 + 207539156903888*2**(2/3)*zeta**8/7920224923939453125 + O(zeta**9)

---

## Taylor expansion $\chi(\zeta)$

In [10]:
χ = ((4 - w**2 * ϕ0**6).expand() / (16 * ζ)).expand()
# display((χ.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(-1, 3)).expand())
display(χ)
# print([src.num2str(χ.coeff(ζ, k)) for k in range(order - 1)])

2**(2/3)/10 + 2**(1/3)*zeta/175 - 32*zeta**2/2625 - 7606*2**(2/3)*zeta**3/3031875 + 43412*2**(1/3)*zeta**4/197071875 + 3997696*zeta**5/6897515625 + 73443616*2**(2/3)*zeta**6/753799921875 - 132580663216*2**(1/3)*zeta**7/7719664999921875 - 248396455936*zeta**8/9190077380859375 - 224428270159801696*2**(2/3)*zeta**9/54530748601323134765625 + O(zeta**10)

---

## Taylor expansion $A_k(\zeta)$ and $B_k(\zeta)$

In [17]:
A, B = taylor_AB(ψ, 3)

In [18]:
for a in A:
    # display(a.subs(ζ, 2 ** sy.Rational(1, 3) * η))
    display(a)

1

-1/225 - 71*2**(2/3)*zeta/77000 + 41*2**(1/3)*zeta**2/73125 + 2623*zeta**3/3898125 + 46432*2**(2/3)*zeta**4/478603125 - 593793623*2**(1/3)*zeta**5/12974226890625 - 564066848*zeta**6/11306998828125 - 379132331724868*2**(2/3)*zeta**7/57704495874416015625 + O(zeta**8)

151439/218295000 + 68401*2**(2/3)*zeta/294525000 - 1796498167*2**(1/3)*zeta**2/8387379000000 - 583721053*zeta**3/1661436562500 - 93484125814411*2**(2/3)*zeta**4/1420418359985625000 + O(zeta**5)

-887278009/2504935125000 - 3032321618951*2**(2/3)*zeta/19417885987500000 + O(zeta**2)

In [19]:
for b in B:
    # display((b.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
    display(b)

2**(1/3)/70 + 2*zeta/225 + 69*2**(2/3)*zeta**2/67375 - 148*2**(1/3)*zeta**3/511875 - 19232*zeta**4/63669375 - 29368*2**(2/3)*zeta**5/797671875 + 1208831912*2**(1/3)*zeta**6/90819588234375 + 49058816*zeta**7/3768999609375 + 207539156903888*2**(2/3)*zeta**8/134643823706970703125 + O(zeta**9)

-1213*2**(1/3)/1023750 - 3757*zeta/2695000 - 3225661*2**(2/3)*zeta**2/13400887500 + 90454643*2**(1/3)*zeta**3/673985812500 + 24359972393*zeta**4/142468185234375 + 2984904217639567*2**(2/3)*zeta**5/115408991748832031250 + O(zeta**6)

16542537833*2**(1/3)/37743205500000 + 115773498223*zeta/162820783125000 + 548511920915149*2**(2/3)*zeta**2/3443438448450000000 + O(zeta**3)

O(1)

---

## Taylor expansion $C_k(\zeta)$ and $D_k(\zeta)$

In [20]:
C, D = taylor_CD(χ, A, B, 3)

In [21]:
for c in C:
    # display((c.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(2, 3)).expand())
    display(c)

2**(2/3)/10 + 2**(1/3)*zeta/50 - 26*zeta**2/7875 - 643*2**(2/3)*zeta**3/433125 - 13568*2**(1/3)*zeta**4/197071875 + 820384*zeta**5/2956078125 + 45690856*2**(2/3)*zeta**6/753799921875 - 46828808*2**(1/3)*zeta**7/12118783359375 - 386324128768*zeta**8/27570232142578125 - 20053558801961008*2**(2/3)*zeta**9/7790106943046162109375 + O(zeta**10)

-947*2**(2/3)/693000 - 947*2**(1/3)*zeta/3465000 + 49213*zeta**2/63063000 + 28974929*2**(2/3)*zeta**3/120607987500 - 21928777841*2**(1/3)*zeta**4/294082476187500 - 22387957645723*zeta**5/148879253569921875 - 110689189547687*2**(2/3)*zeta**6/3804692035675781250 + O(zeta**7)

11192989*2**(2/3)/37110150000 + 11192989*2**(1/3)*zeta/185550750000 - 13568179426889*zeta**2/34648262649000000 - 524855975157853*2**(2/3)*zeta**3/3622318367850000000 + O(zeta**4)

-100443412440047*2**(2/3)/524282921662500000 + O(zeta)

In [22]:
for d in D:
    # display((d.subs(ζ, 2 ** sy.Rational(1, 3) * η)).expand())
    display(d)

1

23/3150 + 1453*2**(2/3)*zeta/693000 - 8878*2**(1/3)*zeta**2/39414375 - 270131*zeta**3/354729375 - 4539842*2**(2/3)*zeta**4/30151996875 + 570560449*2**(1/3)*zeta**5/16966296703125 + 942725500448*zeta**6/16542139285546875 + 4931542652643164*2**(2/3)*zeta**7/519340462869744140625 + O(zeta**8)

-604523/644962500 - 105959543*2**(2/3)*zeta/268017750000 + 34879215863*2**(1/3)*zeta**2/237642405000000 + 5138177650373*zeta**3/13534477597265625 + 1090023854137571*2**(2/3)*zeta**4/12451719389484375000 + O(zeta**5)

2264850139339/5095332742500000 + 940481215440667*2**(2/3)*zeta/3984550204635000000 + O(zeta**2)